In [87]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score

Traceback (most recent call last):
  File "C:\Program Files\JetBrains\PyCharm 2023.2.3\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_trace_dispatch_regular.py", line 408, in __call__
    if not is_thread_alive(t):
           ~~~~~~~~~~~~~~~^^^
  File "C:\Program Files\JetBrains\PyCharm 2023.2.3\plugins\python\helpers\pydev\_pydev_bundle\pydev_is_thread_alive.py", line 18, in is_thread_alive
    return t.isAlive()
           ^^^^^^^^^
AttributeError: '_MainThread' object has no attribute 'isAlive'. Did you mean: 'is_alive'?
Traceback (most recent call last):
  File "C:\Program Files\JetBrains\PyCharm 2023.2.3\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_trace_dispatch_regular.py", line 408, in __call__
    if not is_thread_alive(t):
           ~~~~~~~~~~~~~~~^^^
  File "C:\Program Files\JetBrains\PyCharm 2023.2.3\plugins\python\helpers\pydev\_pydev_bundle\pydev_is_thread_alive.py", line 18, in is_thread_alive
    return t.isAlive()
           ^^^^^^^^^
AttributeError: '_MainT

In [78]:
df = pd.read_csv('tmdb_5000_movies.csv')
print(df.shape)
df.head(3)


(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [79]:
features = ["budget", "popularity", "runtime", "vote_average", "vote_count", "revenue"]
df = df[features].copy()
df[["budget", "revenue"]] = df[["budget", "revenue"]].replace(0, np.nan)
df.dropna()

df['hit'] = (df['revenue'] > df['budget']).astype(int)
print(df['hit'].value_counts(normalize=True))
df.head(3)

hit
1    0.507599
0    0.492401
Name: proportion, dtype: float64


,budget,popularity,runtime,vote_average,vote_count,revenue,hit
0,237000000.0,150.437577,162.0,7.2,11800,2.787965e+09,1
1,300000000.0,139.082615,169.0,6.9,4500,9.610000e+08,1
2,245000000.0,107.376788,148.0,6.3,4466,8.806746e+08,1


In [80]:

X = df[['budget', 'popularity', 'runtime', 'vote_average', 'vote_count']].values
y = df['hit'].values
imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    random_state=42,
    test_size=0.2,
    stratify=y 
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("NaN в X_train_scaled:", np.isnan(X_train_scaled).any())
print("NaN в X_test_scaled:", np.isnan(X_test_scaled).any())

NaN в X_train_scaled: False
NaN в X_test_scaled: False


In [81]:
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)

y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)

In [82]:
class MovieNet(nn.Module):
    def __init__(self, input_dim=5, hidden_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.net(x)
    
model = MovieNet()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)



In [89]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * X_batch.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    
    
    model.eval()
    all_probs = []
    all_true = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            logits = model(X_batch)
            probs = torch.sigmoid(logits)
            all_probs.append(probs.squeeze(1).cpu().numpy())
            all_true.append(y_batch.squeeze(1).cpu().numpy())
    all_probs = np.concatenate(all_probs)
    all_true = np.concatenate(all_true)
    
    preds = (all_probs > 0.8).astype(int)
    
    acc = accuracy_score(all_true, preds)

    try:
        auc = roc_auc_score(all_true, all_probs)
    except ValueError:
        auc = float("nan")  # если вдруг в тесте только один класс

    print(
        f"Epoch {epoch+1}/{num_epochs} | "
        f"train loss: {epoch_loss:.4f} | "
        f"test acc: {acc:.4f} | test AUC: {auc:.4f}"
    )

Epoch 1/100 | train loss: 0.3950 | test acc: 0.7326 | test AUC: 0.9158
Epoch 2/100 | train loss: 0.3949 | test acc: 0.7305 | test AUC: 0.9157
Epoch 3/100 | train loss: 0.3949 | test acc: 0.7378 | test AUC: 0.9159
Epoch 4/100 | train loss: 0.3950 | test acc: 0.7378 | test AUC: 0.9157
Epoch 5/100 | train loss: 0.3949 | test acc: 0.7336 | test AUC: 0.9158
Epoch 6/100 | train loss: 0.3949 | test acc: 0.7347 | test AUC: 0.9156
Epoch 7/100 | train loss: 0.3949 | test acc: 0.7336 | test AUC: 0.9157
Epoch 8/100 | train loss: 0.3947 | test acc: 0.7378 | test AUC: 0.9159
Epoch 9/100 | train loss: 0.3948 | test acc: 0.7336 | test AUC: 0.9160
Epoch 10/100 | train loss: 0.3947 | test acc: 0.7357 | test AUC: 0.9157
Epoch 11/100 | train loss: 0.3947 | test acc: 0.7326 | test AUC: 0.9159
Epoch 12/100 | train loss: 0.3952 | test acc: 0.7326 | test AUC: 0.9159
Epoch 13/100 | train loss: 0.3949 | test acc: 0.7326 | test AUC: 0.9152
Epoch 14/100 | train loss: 0.3970 | test acc: 0.7326 | test AUC: 0.9151
E